In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import utils
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim

In [2]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


In [6]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [7]:
def ToPos(turn, bias_x, bias_y):
    letter = ord(turn[0]) - ord('a')
    if 0 <= letter + bias_x <= 14:
        letter += bias_x
    else:
        return -1, -1
    
    num = int(turn[1:]) - 1
    if 0 <= num + bias_y <= 14:
        num += bias_y
    else:
        return -1, -1

    return letter, num

def Transform(game, color, turn, bias):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard

    res = []
    labels = []

    for bias_x in range(-bias, bias + 1):
        for bias_y in range(-bias, bias + 1):
            curPlayer = 1
            whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
            blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
            for i in range(1, turn + 1):
                if len(game[i]) < 2:
                    continue
                CurTurn = toTurn(game[i], bias_x, bias_y)
                if CurTurn[0] == -1 and CurTurn[1] == -1:
                    continue
                if curPlayer == 1:
                    blackBoard[CurTurn] = 1
                else:
                    whiteBoard[CurTurn] = -1
                curPlayer *= -1
            
            curBoard = np.zeros(shape=(3, 15, 15))
            curBoard[0,:] = playerBoard
            curBoard[1,:] = blackBoard
            curBoard[2,:] = whiteBoard
            label = toTurn(game[turn + 1], bias_x, bias_y)
            if label[0] == -1 and label[1] == -1:
                continue
            
            res.append(curBoard)
            labels.append(label[0] * 15 + label[1])

    return np.asarray(res), np.asarray(labels)

In [8]:
NumOfFine = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        NumOfFine += 1

In [9]:
Dataset = np.ndarray(shape=NumOfFine, dtype='U1000')
idx = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        Dataset[idx] = elem
        idx += 1
print(Dataset.shape)

(1941123,)


In [10]:
def check(Board, turn):
    Board *= turn
    BoardUp = np.ndarray(shape=(15, 15), dtype=np.int8)
    BoardLeft = np.ndarray(shape=(15, 15), dtype=np.int8)
    BoardDiag1 = np.ndarray(shape=(15, 15), dtype=np.int8)
    BoardDiag2 = np.ndarray(shape=(15, 15), dtype=np.int8)
    
    for i in range(Board.shape[0]):
        for j in range(Board.shape[1]):
            if i > 0:
                BoardUp[i, j] = BoardUp[i - 1, j] + Board[i, j]
                if i - 4 >= 0 and BoardUp[i, j] - BoardUp[i - 4, j] + Board[i - 4, j] == 4:
                    for k in range(5):
                        if Board[i - k, j] == 0:
                            Board *= turn
                            return i - k + 1, j + 1
            if j > 0:
                BoardLeft[i, j] = BoardLeft[i, j - 1] + Board[i, j]
                if j - 4 >= 0 and BoardLeft[i, j] - BoardLeft[i, j - 4] + Board[i, j - 4] == 4:
                    for k in range(5):
                        if Board[i, j - k] == 0:
                            Board *= turn
                            return i + 1, j - k + 1
            if i > 0 and j > 0:
                BoardDiag1[i, j] = BoardDiag1[i - 1, j - 1] + Board[i, j]
                if i-4 >= 0 and j-4 >= 0 and BoardDiag1[i, j] - BoardDiag1[i - 4, j - 4] + Board[i - 4, j - 4] == 4:
                    for k in range(5):
                        if Board[i - k, j - k] == 0:
                            Board *= turn
                            return i - k + 1, j - k + 1
            if i > 0 and j < 14:
                BoardDiag2[i, j] = BoardDiag2[i - 1, j + 1] + Board[i, j]
                if i-4 >= 0 and j+4 < 15 and BoardDiag2[i, j] - BoardDiag2[i - 4, j + 4] + Board[i - 4, j + 4] == 4:
                    for k in range(5):
                        if Board[i - k, j + k] == 0:
                            Board *= turn
                            return i - k + 1, j + k + 1
    
    Board *= turn
    return -1, -1

In [11]:
Board = np.array([
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, -1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        ])

print(check(Board, 1))

(11, 4)


In [12]:
def GetRandom(data, bias):
    game = []
    while len(game) == 0:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2, bias)
    else:
        board, label = toBoards(game, color, turn * 2 + 1, bias)

    return board, label

def Generator(size, data):
    Batch = np.ndarray(shape=(0, 3, 15, 15), dtype=np.int8)
    BatchLabels = np.ndarray(shape=0, dtype=np.long)
    for i in range(size):
        res, labels = GetRandom(data, 1)
        Batch = np.concatenate((Batch, res), axis=0)
        BatchLabels = np.concatenate((BatchLabels, labels), axis=0)

    return torch.from_numpy(Batch).float(), torch.from_numpy(BatchLabels)

In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(16, 1, kernel_size=5, stride=1, padding=2),
            nn.ReLU())

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(-1, 225)
        
        return x

In [15]:
device = torch.device('cpu')

net = ConvNet()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
CntBatch = 1000000
CntEpoch = 2
BatchSize = 16
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [184]:
for epoch in range(CntEpoch):
    running_loss = 0.0
    
    for i in range(CntBatch):
        inputs, labels = Generator(BatchSize, Dataset)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('Finished Training')

[1,   200] loss: 4.450


KeyboardInterrupt: 

In [18]:
device = torch.device('cpu')
net.load_state_dict(torch.load("model4.dms", map_location=device))

In [19]:
correct = 0
total = 0
for i in range(100000):
    if i % 100 == 99:
        print('Accuracy of the network on the 100 positions: %5f %%' % (100 * correct / total))    
    inputs, labels = Generator(BatchSize, Dataset)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 100000 positions: %d %%' % (100 * correct / total))

Accuracy of the network on the 1000 positions: 41.075193 %
Accuracy of the network on the 1000 positions: 41.821809 %


KeyboardInterrupt: 